Iterators, Generators and Classic Coroutines

In [10]:
import sentence
from sentence import Sentence
s = Sentence('"The time has come," the Walrus said,')

In [15]:
for words in s:
    print(words,sep='\t')

list(s)
s[0]
s[5]
s[-1]

The
time
has
come
the
Walrus
said


'said'

Why Sequences Are Iterable: The iter Function

In [1]:
#Whenever Python needs to iterate over an object x, it automatically calls iter(x).

#The iter built in function:

#1 Checks whether the object implements __iter__ and calls that to obtain an iterator.

#2 If __iter__ is not implemented but __getitem__ is then iter() creates an iterator that tries to fetch items by index starting form 0

#3 If that fails, Python raised TypeError usually sayign C object is not iterable where c is the class of the target object.

Imp

In [2]:
#Hence clearly an object is not only considered iterable when it necessarily inherits the iter method but also iterable if it only inherits the getitem method.

In [3]:
#Hence all the sequnce objects have either the iter or the getitem method

In [ ]:
class Spam:
    def __getitem__(self, i):
        print('->', i)
        raise IndexError()


spam_can  =Spam()
iter(spam_can)
list(spam_can)

#checking if iterable

from collections import abc
isinstance(spam_can, abc.Iterable)
class GooseSpam:
    def __iter__(self):
        pass
from collections import abc
issubclass(GooseSpam, abc.Iterable)
goose_spam_can = GooseSpam()
isinstance(goose_spam_can, abc.Iterable)

#Imp

In [21]:
#the most accurate way to check whether an object x is iterable is to call iter(x) and handle a TypeError exception if it isnt. THis is more accurate than using isinstance and issubclass because iter(x) aso considers the legacy __getitem__ method while Iterable ABC does not.


In [ ]:
#Using iter with a Callable